# loader

> Module for associating data on the disk with database entries to be queried for training and evaluation.

In [2]:
#| default_exp loader

In [3]:
#| hide 
%load_ext autoreload
%autoreload 2

In [4]:
#| hide
from nbdev.showdoc import *

import csv
from enum import Enum
from typing import List

from sqlalchemy import Boolean, Column, Integer, String, create_engine, text
from sqlalchemy.orm import declarative_base, sessionmaker

from enum import Enum, auto
from typing import Dict, List, Optional, Tuple, Union

from pathlib import Path


In [5]:
#| export

Base = declarative_base()

class StudyRecord(Base):
    """
    Class modeling the records in our sqlite database.

    The attributes of this class must match the column names in NavySleepResearch/data_sets.csv. Using
    Python's built-in CSVReader class, we get an array of dicts that are easy to turn into StudyRecord
    objects via dict unpacking (** operator) and kwarg matching, which is a fancy way to say:
      new_record = StudyRecord(**csv_row_dict)
    """

    __tablename__ = "data_sets"

    id = Column(
        Integer, primary_key=True
    )  # database id column, not id of patient within study

    # Identification of patient and data set
    subject_id = Column(String)
    data_set_name = Column(String)

    # Patient metadata
    age = Column(Integer)
    sex = Column(Integer)  # Integer: maps to Enum
    race = Column(Integer)  # Integer: maps to Enum
    has_apnea = Column(Boolean)
    has_idiopathic_hypersomnia = Column(Boolean)
    has_sleep_paralysis = Column(Boolean)
    has_restless_leg_syndrome = Column(Boolean)
    has_insomnia = Column(Boolean)
    has_parasomnia = Column(Boolean)
    has_rem_sleep_disorder = Column(Boolean)
    has_narcolepsy = Column(Boolean)
    has_nocturia = Column(Boolean)

    # Identify what data is available and where it is
    psg_location = Column(String)
    accelerometer_location = Column(String)
    heartrate_location = Column(String)
    activity_location = Column(String)

    def __repr__(self) -> str:
        return f"{self.subject_id}:{self.data_set_name}"

    def diagnosis(self) -> str:
        known_conditions = [
            "apnea",
            "idiopathic_hypersomnia",
            "sleep_paralysis",
            "restless_leg_syndrome",
            "insomnia",
            "parasomnia",
            "rem_sleep_disorder",
            "narcolepsy",
            "nocturia",
        ]

        self_dict = (
            self.__dict__
        )  # lets us paste known_conditions into dict keys for easy iteration

        found_conditions = ""

        for condition in known_conditions:
            if self_dict[f"has_{condition}"]:
                found_conditions += f" -- {condition.replace('_', ' ')}\n"
        return found_conditions


class DatabaseLiaison:
    def __init__(self, db_filename: str):
        self.engine = create_engine(f"sqlite:///{db_filename}")
        self.Session_maker = sessionmaker(bind=self.engine)
        Base.metadata.create_all(
            bind=self.engine
        )  # makes the actual tables in database, if they don't exist

    def add(self, record: StudyRecord):
        session = self.Session_maker()
        session.add(record)
        session.commit()

    def add_all(self, records: List[StudyRecord]):
        session = self.Session_maker()
        session.add_all(records)
        session.commit()

    def select(self, selector: str = "") -> List[StudyRecord]:
        """
        Runs raw SQL on database; selector adds a WHERE clause to SELECT * FROM data_sets
        The default "" selects everything, equivalent to '''SELECT * FROM data_sets'''
        :param selector: WHERE clause contents
        :return: List of study records matching the selector.
        """
        session = self.Session_maker()
        return session.query(StudyRecord).filter(text(selector)).all()

    def to_csvDictWriterDicts(self) -> List[dict]:
        """
        Converts database to a list of dicts ready to be sent into a CSV by csv.DictWriter.
        :return: `[dict]` one per row with keys being the columns.
        """
        session = self.Session_maker()
        records = [r.__dict__ for r in session.query(StudyRecord).all()]

        for r in records:
            # pop those keys that are not part of CSV
            r.pop("id")  # db PK, dont persist this
            r.pop("_sa_instance_state")  # artifact of SQLAlchemy class.

        return records

    def to_csv(self, filename: str):
        records = self.to_csvDictWriterDicts()
        with open(filename, "w", newline="") as csvfile:
            # This must be hardcoded to ensure deterministic order, AFAIK
            fieldnames = [
                "subject_id",
                "data_set_name",
                "age",
                "sex",
                "race",
                "has_apnea",
                "has_idiopathic_hypersomnia",
                "has_sleep_paralysis",
                "has_restless_leg_syndrome",
                "has_insomnia",
                "has_parasomnia",
                "has_rem_sleep_disorder",
                "has_narcolepsy",
                "has_nocturia",
                "psg_location",
                "accelerometer_location",
                "heartrate_location",
                "activity_location",
            ]

            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for rec in records:
                writer.writerow(rec)


In [6]:
#| hide
import nbdev; nbdev.nbdev_export()